## 제 1유형

1-1. 다음은 Boston Housing 데이터셋이다. 데이터 중 컬럼들의 결측값을 전부 제거 후 데이터를 처음부터 순서대로 70%를 추출하여 변수 중 'tax'의 사분위수 Q1값을 구하시오.

In [20]:
import pandas as pd
a = pd.read_csv('3rd_boston_housing.csv')
b = a.dropna()
#b.isnull().sum()
c = b[:int(len(b) * 0.7)]
answer = c['tax'].describe()['25%']

print(answer)

264.0


1-2. 다음은 국가별 국내 입국자 수 데이터이다. 2000년도 전체 입국자 평균보다 국내에 많이 입국한 국가의 수를 산출하시오.

In [32]:
import pandas as pd
data = pd.read_csv('3rd_tour.csv', index_col = 'year', encoding = 'euc-kr')
mean = data.loc[2000].mean()

print(sum(data.loc[2000] > mean))

7


1-3. 다음 데이터셋의 컬럼 중 빈값 또는 결측치를 확인하여 결측치의 비율이 높은 변수명을 출력하시오.

In [42]:
import pandas as pd
a = pd.read_csv('3rd_titanic.csv')
a.isnull().sum()

b = a.isnull().sum()/len(a) * 100
#print(b)

print(b.sort_values(ascending = False).index[0])

Age


## 제 2유형

아래 여행객 보험 데이터셋을 참조하여 여행객의 정보들을 기반으로 여행보험 상품 가입 여부 예측하시오.(ID와 예측치를 csv파일로 저장하여 제출하시오.)

In [1]:
import pandas as pd

test = pd.read_csv('3rd_TravelInsurancePrediction_test.csv')
train = pd.read_csv('3rd_TravelInsurancePrediction_train.csv')
y = train['TravelInsurance']
train = train.drop(['TravelInsurance'], axis = 1)

#ID 저장
test_id = test['ID']
test = test.drop(['ID'], axis = 1)
train = train.drop(['ID'], axis = 1)

#결측치 확인
#결측치 없음
# train.isnull().sum()
# test.isnull().sum()

#라벨인코더(Employment Type, GraduateorNot, FrequentFlyer, EverTravelledAbroad)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['Employment Type'] = le.fit_transform(train['Employment Type'])
train['GraduateOrNot'] = le.fit_transform(train['GraduateOrNot'])
train['FrequentFlyer'] = le.fit_transform(train['FrequentFlyer'])
train['EverTravelledAbroad'] = le.fit_transform(train['EverTravelledAbroad'])

test['Employment Type'] = le.fit_transform(test['Employment Type'])
test['GraduateOrNot'] = le.fit_transform(test['GraduateOrNot'])
test['FrequentFlyer'] = le.fit_transform(test['FrequentFlyer'])
test['EverTravelledAbroad'] = le.fit_transform(test['EverTravelledAbroad'])

#MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

#데이터 분할
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train, y, test_size = 0.2, random_state = 49)
# print(x_train.shape, x_valid.shape, y_train.shape, y_valid.shape)

#모델링
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state = 1000, n_estimators = 100, max_depth = 5, min_samples_leaf = 5)
model.fit(x_train, y_train)
predict = model.predict_proba(x_valid)

#roc_auc_score
# from sklearn.metrics import roc_auc_score
# print(roc_auc_score(y_valid, predict[:,1]))

#테스트데이터
predict_test = model.predict_proba(test)
submit = pd.DataFrame({'ID' : test_id, 'TravelInsurance' : predict_test[:,1]})
submit.to_csv('3회.csv', index = False)